In [ ]:
!pip install -q transformers datasets evaluate huggingface_hub peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade --force-reinstall datasets

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.3.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pyarrow-20.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.32.4-py3-none-any.whl.metadata (14 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp31

In [ ]:
from datasets import load_dataset

# Load AG News dataset for a clearer fine-tuning improvement
dataset = load_dataset("ag_news")
dataset = dataset.shuffle(seed=42)
print(dataset)

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [ ]:
!pip list

Package                               Version
------------------------------------- ------------------
absl-py                               1.4.0
accelerate                            1.7.0
aiofiles                              24.1.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.12.11
aiosignal                             1.3.2
alabaster                             1.0.0
albucore                              0.0.24
albumentations                        2.0.8
ale-py                                0.11.1
altair                                5.5.0
annotated-types                       0.7.0
antlr4-python3-runtime                4.9.3
anyio                                 4.9.0
argon2-cffi                           25.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.2
arviz                                 0.21.0
astropy                               7.1.0
astropy-iers-data                     0.2025.6.2.0.38

In [ ]:
print("Sample News:", dataset["train"][0]["text"])
print("Label (0 = World, 1 = Sports, 2 = Business, 3 = Sci/Tech):", dataset["train"][0]["label"])

Sample News: Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.
Label (0 = World, 1 = Sports, 2 = Business, 3 = Sci/Tech): 0


In [ ]:
from datasets import DatasetDict

# Reduce size for faster demo
small_train = dataset["train"].select(range(4000))
small_test = dataset["test"].select(range(1000))

# Train/Validation Split
small_train_test = small_train.train_test_split(test_size=0.2, seed=42)
datasets = DatasetDict({
    "train": small_train_test["train"],
    "validation": small_train_test["test"],
    "test": small_test
})
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3200
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets["train"][1])
#1 indicates pay attention to that token and 0 indicates do not pay attention

{'text': 'Levet willing to miss out Thomas Levet says he would be willing to be left out of the foursomes and fourballs at the Ryder Cup as long as the European team wins.', 'label': 1, 'input_ids': [101, 23310, 3388, 5627, 2000, 3335, 2041, 2726, 23310, 3388, 2758, 2002, 2052, 2022, 5627, 2000, 2022, 2187, 2041, 1997, 1996, 23817, 8462, 2015, 1998, 2176, 18510, 2012, 1996, 11731, 2452, 2004, 2146, 2004, 1996, 2647, 2136, 5222, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from transformers import AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
target_modules = ["query", "value"]  # key layers in attention blocks
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 297,988 || all params: 109,783,304 || trainable%: 0.2714


In [1]:
# LoRA rank: defines the dimensionality of the low-rank adapter matrices.
# A higher rank increases model capacity but also adds more trainable parameters.

# LoRA alpha: scaling factor for the LoRA output.
# The actual adapter output is scaled by (alpha / rank) to balance with the frozen base model.

# Target modules: list of specific layer names in the model where LoRA adapters should be injected.
# Typically includes attention-related layers like "q_proj" or "v_proj" in transformer models.

# LoRA dropout: dropout probability applied only to the LoRA adapter output during training.
# Helps reduce overfitting, especially on smaller datasets.

# Bias setting: controls whether and where to train bias terms.
# Options: "none" (do not train any biases), "all" (train all biases), "lora_only" (train only in layers with LoRA).

# Task type: specifies the kind of downstream task for which the model is being fine-tuned.
# Examples: SEQ_CLS (sequence classification), CAUSAL_LM (language modeling), TOKEN_CLS (token classification), etc.

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
print(pipe("The stock market closed higher today after positive earnings reports."))
print(pipe("The player scored a hat trick in the championship game."))

Device set to use cuda:0


[{'label': 'LABEL_2', 'score': 0.4248039722442627}]
[{'label': 'LABEL_2', 'score': 0.3835366368293762}]


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",                        # Directory to save model checkpoints and outputs
    eval_strategy="epoch",                  # Evaluate at the end of every epoch
    save_strategy="epoch",                        # Save checkpoint at the end of every epoch
    learning_rate=2e-4,                            # Learning rate for optimizer
    per_device_train_batch_size=8,                 # Batch size per device (GPU/TPU) during training
    per_device_eval_batch_size=8,                  # Batch size per device during evaluation
    num_train_epochs=2,                            # Total number of training epochs
    weight_decay=0.01,                             # Weight decay to apply (for regularization)
    logging_dir="./logs",                         # Directory to store logs
    push_to_hub=False,                             # Whether to push model to Hugging Face Hub
    report_to="none"                               # Disable logging to external tools like WandB/Comet
)

In [ ]:
# Load Hugging Face's built-in evaluation library
import evaluate
import numpy as np

# Load the "accuracy" metric — this will download a standard implementation for accuracy scoring
accuracy = evaluate.load("accuracy")

# Define a custom compute_metrics function that the Trainer will use during evaluation
def compute_metrics(eval_pred):
    # eval_pred is a tuple: (model_outputs, true_labels)
    logits, labels = eval_pred

    # Convert logits (raw model outputs) into predicted class indices
    # The model outputs a vector of scores for each class (e.g., [0.2, 0.8])
    # np.argmax selects the index with the highest score, i.e., the predicted label
    predictions = np.argmax(logits, axis=-1)

    # Compare predicted labels with true labels and calculate accuracy
    # accuracy.compute returns a dictionary, e.g., {"accuracy": 0.89}
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-37-f9683c19c83d>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.371083,0.907500
2,0.278100,0.380552,0.906250


TrainOutput(global_step=800, training_loss=0.26576786041259765, metrics={'train_runtime': 226.5473, 'train_samples_per_second': 28.25, 'train_steps_per_second': 3.531, 'total_flos': 1689799675084800.0, 'train_loss': 0.26576786041259765, 'epoch': 2.0})

In [ ]:
results = trainer.evaluate(tokenized_datasets["test"])
print(results)

{'eval_loss': 0.3923862874507904, 'eval_accuracy': 0.906, 'eval_runtime': 15.0549, 'eval_samples_per_second': 66.424, 'eval_steps_per_second': 8.303, 'epoch': 2.0}


In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer)
#0 = World, 1 = Sports, 2 = Business, 3 = Sci/Tech
print(pipe("The stock market closed higher today after positive earnings reports."))
print(pipe("The player scored a hat trick in the championship game."))

Device set to use cuda:0


[{'label': 'LABEL_2', 'score': 0.9903820157051086}]
[{'label': 'LABEL_1', 'score': 0.9910256266593933}]


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/saideepkoppaka/results/commit/dc60046128da9febe0d9fb0e2e5729866651066d', commit_message='End of training', commit_description='', oid='dc60046128da9febe0d9fb0e2e5729866651066d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/saideepkoppaka/results', endpoint='https://huggingface.co', repo_type='model', repo_id='saideepkoppaka/results'), pr_revision=None, pr_num=None)